## Alisa Todorova

In [1]:
from sage.all import *

The following code generates an RSA key with a modulus N of n bits, generates a random polynomial: 

f(x)=x^2+ax+b mod N

with a small root |x_0| < 2^(n/3), and recovers this root using Coppersmith’s technique.

In [2]:
def keyGen(n=256):
  "Generates an RSA key"
  while True:
    p=random_prime(2^(n//2));q=random_prime(2^(n//2));e=3
    if gcd(e,(p-1)*(q-1))==1: break
  d=inverse_mod(e,(p-1)*(q-1))
  Nn=p*q
  print("p=",p,"q=",q)
  print("N=",Nn)
  print("Size of N:",Nn.nbits())
  return Nn,p,q,e,d

"Finds a small root of polynomial x^2+ax+b=0 mod N"
def CopPolyDeg2(a,b,Nn):
  n=Nn.nbits()
  X=2^(n//3-3)
  M=matrix(ZZ,[[X^2,a*X,b],\
               [0  ,Nn*X,0],\
               [0  ,0  ,Nn]])
  V=M.LLL()
  v=V[0]
  R.<x> = ZZ[]
  f=sum(v[i]*x^(2-i)/X^(2-i) for i in range(3))
  return f.roots()

def test():
  """Generates a random polynomial with a small root x0 modulo Nn
     and recovers his root."""
  Nn,p,q,e,d=keyGen()
  n=Nn.nbits()
  x0=ZZ.random_element(2^(n//3-10))
  a=ZZ.random_element(Nn)
  b=mod(-x0^2-a*x0,Nn)
  print("x0=",x0)
  print(CopPolyDeg2(a,b,Nn))

test()

p= 81148965068916712460696931040507301759 q= 42635304019031611976549643573856799573
N= 3459810796543040601932605684167873238508415479328158216653323046757493348907
Size of N: 251
x0= 3390946262337443979873
[(3390946262337443979873, 1), (-2310060546572452968656848621531895210562013071392/52074820704467544387439459, 1)]


### 1.3 Polynomials of degree 3
Modify the previous code to find small roots of polynomials of degree 3. What is the new bound on x_0?

In [3]:
"Finds a small root of polynomial x^3+ax^2+bx+c=0 mod N"
def CopPolyDeg3(a,b,c,Nn):
  n=Nn.nbits()
  X=2^(n//6-3) # (n//6-3) ensures that X^3 is smaller than Nn
  M=matrix(ZZ,[[X^3,a*X^2,b*X,c],\
               [0  ,Nn*X^2,0,0],\
               [0  ,0  ,Nn*X,0],\
               [0  ,0  ,0,Nn]])
  V=M.LLL()
  v=V[0]
  R.<x> = ZZ[]
  f=sum(v[i]*x^(3-i)/X^(3-i) for i in range(4))
  return f, f.roots()

In [4]:
def test():
    """Generates a random polynomial with a small root x0 modulo Nn
    and recovers his root."""
    Nn, p, q, e, d = keyGen()
    n = Nn.nbits()
    x0 = ZZ.random_element(2 ^ (n // 6 - 10)) # (n//6-10) ensures that x0^3 is smaller than Nn
    a = ZZ.random_element(Nn)
    print("a=", a)
    b = ZZ.random_element(Nn)
    print("b=", b)
    c = mod(-x0 ^ 3 - a * x0 ^ 2 - b * x0, Nn)
    print("c=", c)
    print("x0=", x0)
    polynomial, roots = CopPolyDeg3(a, b, c, Nn)
    print("polynomial=", polynomial)
    print("roots=", roots)


test()

p= 339689338561372054374259301840940987371 q= 198833485600304095938975144179941826501
N= 67541615207419393238036369990185131302570648295314518369434899472092114118871
Size of N: 256
a= 49660948123930424666816797036933573882480867484011968552171338390244175019298
b= 50831768301464795727379908330605632822656657824128866997855614468535412237016
c= 10687140017733583282995761829639332364647973239570874920662960500552103027656
x0= 1061616952
polynomial= 1644759938011396594417577155442226664128*x^3 + 1140059011853963613441256809151418487994275290873198*x^2 - 395544242119092309198587024567839683127871658057441838312124215*x + 418629623447576707192840313580486682332572153871170437902835347584912264
roots= [(1061616952, 1)]


### 1.4 Application to breaking RSA encryption with small exponent e

Let

N = 709346188783707808067985067028456920198443445150943868007539748700843008346940940287222002891029537499396908056848095671838771377752278390679951948808475707450073714577930152529057057137554724537719912515486712408422541711159469388603174949214558211229232753033853223644282969544939436208933732047182752629

be an RSA modulus of size 1017 bits. Let m be a message with m < 2^168. Let c = (2^1016 + m)^3 mod N
with

c = 106931790228306983984681105468768847489271870726744288161524001625441564428928138082764847546330455645721020672129383569332864980247631047991368975076235508102167034128121821366927744907882617928973322081585262275454789559141797135693781509609801881376951044409238368040773136628352506077437800520604807524

Recover the message m using Coppersmith’s technique.



In [5]:
# The following code is based on slide 9 of lecture slides "The RSA cryptosystem, Part 2: attacks against RSA"

# Coppersmith's technique to find roots of f(x) modulo N
def coppersmith(N,c):
    B = 2^1016
    k = 0  # Chosen arbitrarily

    # Polynomial f(x) = (B * 2^k + x)^3 - c (mod N)
    R.<x> = PolynomialRing(Zmod(N))
    f = (B * 2^k + x)^3 - c

    roots = f.small_roots(X=2^168)

    return roots

# Given parameters
N = 709346188783707808067985067028456920198443445150943868007539748700843008346940940287222002891029537499396908056848095671838771377752278390679951948808475707450073714577930152529057057137554724537719912515486712408422541711159469388603174949214558211229232753033853223644282969544939436208933732047182752629
c = 106931790228306983984681105468768847489271870726744288161524001625441564428928138082764847546330455645721020672129383569332864980247631047991368975076235508102167034128121821366927744907882617928973322081585262275454789559141797135693781509609801881376951044409238368040773136628352506077437800520604807524

message = coppersmith(N,c)

print("Recovered message:", message[0])


Recovered message: 246444312520370940658372328078267298078392115432165
